In [13]:
import pandas as pd
from fuzzywuzzy import process
from statsbombpy import sb
from rapidfuzz import process, fuzz

# Leer datos de Excel
df_excel = pd.read_excel('Euro_2024.xlsx',sheet_name='Player_euro')

# Leer datos de StatsBomb
events_euro = sb.events(match_id=3942349)
# Filtrar datos de StatsBomb para eliminar filas con NaN en las columnas relevantes
events_euro = events_euro.dropna(subset=['player', 'player_id'])


c:\Users\thermaltaque\OneDrive - Universidad Tecnologica del Peru\Khalil\Data analytics\statsbombpy\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [14]:
def preprocesar_nombre(nombre):
    return nombre.lower().strip()

df_excel['Player'] = df_excel['Player'].apply(preprocesar_nombre)
events_euro['player'] = events_euro['player'].apply(preprocesar_nombre)

In [15]:
def encontrar_mejor_coincidencia(nombre_excel, nombres_statsbomb, ids_statsbomb):
    mejor_coincidencia = process.extractOne(nombre_excel, nombres_statsbomb, scorer=fuzz.ratio)
    if mejor_coincidencia:
        nombre_mas_parecido, similitud = mejor_coincidencia[:2]
        if similitud > 80:  # Puedes ajustar este umbral según tus necesidades
            return ids_statsbomb[nombres_statsbomb.index(nombre_mas_parecido)]
    return None

In [16]:
nombres_statsbomb = events_euro['player'].tolist()
ids_statsbomb = events_euro['player_id'].tolist()

df_excel['player_id'] = df_excel['Player'].apply(lambda x: encontrar_mejor_coincidencia(x, nombres_statsbomb, ids_statsbomb))

In [17]:
df_excel.to_excel('archivo_excel_actualizado.xlsx', index=False)